<a href="https://colab.research.google.com/github/kangmg/PySCF4ASE/blob/main/test_notebooks/PySCF_ase_wfn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile requirements.txt
ase
pyscf
gpu4pyscf-cuda12x # 11 or 12 버전 확인 필요
cutensor-cu12 # 11 or 12 버전 확인 필요
torch

Writing requirements.txt


In [2]:
%pip install -q -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.0/385.0 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.5/124.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.9/156.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.3/165.3 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 23.5 MB/s eta 0:00:00


In [16]:
#@title pyscf calculator : wfn method / mp2, ccsd


'''
===================================================
TODO
---------------------------------------------------

[ ] default 값 세팅하기
[ ] system_changes 의미
[ ] init에서 **kwargs 말고 명시적으로 추가
[ ] dm21도 사용 가능하게 하기 !! 가능하다면 !!
[ ] 데코레이터 역할 알아보고 추가하기
[ ] geometric pyscf랑 성능 벤치마크 해보기
[ ] free energy를 넣는 이유? vib 때매 그러나.. 이외에 ase에서 vib 결과 보는거 더 공부해보기
[ ] warning msg 해결
# /usr/local/lib/python3.10/dist-packages/pyscf/gto/mole.py:1284: UserWarning: Function mol.dumps drops attribute ctxlock because it is not JSON-serializable warnings.warn(msg)
[ ] 버려지는 값들도 접근 가능하게 추가
# pyscf.dft.RKS가 반환하는 다른 값들
#     mo_energy :
#         Orbital energies
#     mo_occ
#         Orbital occupancy
#     mo_coeff
#         Orbital coefficients
===================================================



[Units]
=======  ===========  =============
Program  Quantities   Unit
-------  -----------  -------------
PySCF    Energy       Ha
PySCF    Force        Ha/Bohr
ASE      Energy       eV
ASE      Force        eV/Ang
=======  ===========  =============
'''

import numpy as np
from ase.calculators.calculator import Calculator, all_changes
from ase.units import Ha, Bohr
import torch
#import pyscf
#import gpu4pyscf

class PySCFCalculator(Calculator):
    '''
    Description
    ===========
    PySCF ASE Calculator for post-HF( MP2, CCSD(T) ) caculations
    '''

    '''
    ===============
    Units
    ---------------
    Energy  eV
    ===============
    '''


    implemented_properties = ['energy']

    default_parameters = {
        'charge': 0, # system charge
        'spin': 0, # (= nelec alpha-beta = 2S)
        'method': 'mp2', # mp2, ccsd
        'hf': 'rhf', # uhf or rhf
        'symmetry': False, # point group symmetry : e.g. Cs or C2v Ref. https://github.com/pyscf/pyscf/blob/master/examples/gto/13-symmetry.py
        'basis': 'def2-tzvp', # basis set
        'density_fit': True, # resolution of identity approximation
        'auxbasis': 'auto', # basis for density_fit : e.g. 'cc-pvdz-jkfit' or 'auto'(pyscf sets automatically)
        'device': 'auto', # gpu, cpu
        'max_cycle': 50, # max number of iterations
        'conv_tol': 1e-9, # converge threshold
        'verbose' : 4, # output log level
        'max_memory': 150000, # MB unit
        'chkfile': None, # chkpoint file contains MOs, orbital energies etc. : (str) e.g. './checkpoint/pyscf.chk'
        'output': None, # log file : (str) e.g. './output/pyscf_output.log'
        # TODO : conv_tol_grad, init_guess
    }

    def _set_device(self):
        # set device
        if self.parameters.device == 'auto':
            self.parameters.device = 'gpu' if torch.cuda.is_available() else 'cpu'

        if self.parameters.device not in ["gpu", "cpu"]:
            raise ValueError(f"Invalid device: {self.parameters.device}, 'gpu' or 'cpu' supported.")

    def _set_post_HF_method(self):
        """set post-HF method
        """
        # cpu --> pyscf
        if self.parameters.device == 'cpu':
            import pyscf
            self.scf = pyscf.scf
            # mp2
            if self.parameters.method == 'mp2':
                self.post_hf = pyscf.mp.MP2
            # ccsd
            elif self.parameters.method == 'ccsd':
                self.post_hf = pyscf.cc.CCSD

        # gpu --> gpu4pyscf
        elif self.parameters.device == 'gpu':
            import gpu4pyscf
            self.scf = gpu4pyscf.scf
            # with density_fit
            if self.parameters.density_fit:
                # mp2 with RI approx.
                if self.parameters.method == 'mp2':
                    from gpu4pyscf.mp.dfmp2 import DFMP2
                    self.post_hf = DFMP2
                # CCSD with RI approx. : not yet implemented
                elif self.parameters.method == 'ccsd':
                    raise NotImplementedError("gpu4pyscf.cc.ccsd_incore.CCSD does not support density_fit yet.")
            # without density_fit
            elif not self.parameters.density_fit:
                if self.parameters.method == 'mp2':
                    from gpu4pyscf.mp.mp2 import MP2
                    self.post_hf = MP2
                elif self.parameters.method == 'ccsd':
                    from gpu4pyscf.cc.ccsd_incore import CCSD
                    self.post_hf = CCSD


    def __init__(self, restart=None, label='PySCF', **kwargs):
        super().__init__(restart=restart, label=label, **kwargs)
        self.mol = None
        self.mf = None

    def calculate(self, atoms=None, properties=['energy'], system_changes=all_changes):
        super().calculate(atoms, properties, system_changes)

        self._set_device()

        if self.atoms is None:
            raise ValueError("atoms object is not set.")

        if system_changes:
            self.results.clear()
            self.mol = None
            self.mf = None

        if self.mol is None:
            self._generate_molecule()

        if self.mf is None:
            self._run_post_HF()

        if 'energy' in properties and 'energy' not in self.results:
            # pyscf energy unit : Hatree
            # ase energy unit : eV
            self.results['energy'] = (self.e_hf + self.e_corr) * Ha

    def _generate_molecule(self):
        positions = self.atoms.get_positions()
        symbols = self.atoms.get_chemical_symbols()
        atom_str = "; ".join([f"{s} {p[0]} {p[1]} {p[2]}" for s, p in zip(symbols, positions)])
        self.mol = pyscf.M(atom=atom_str,
                         basis=self.parameters.basis,
                         charge=self.parameters.charge,
                         spin=self.parameters.spin,
                         symmetry=self.parameters.symmetry,
                         verbose = self.parameters.verbose,
                         max_memory=self.parameters.max_memory,
                         output=self.parameters.output,
                         unit='Angstrom')
    def _hf(self, mol):
        if self.parameters.hf == 'rhf':
            return self.scf.RHF(mol)
        elif self.parameters.hf == 'uhf':
            return self.scf.UHF(mol)

    def _run_post_HF(self):

        # post HF 설정 : MP2, CCSD  --> self.post_hf
        self._set_post_HF_method()

        self.mf = self._hf(self.mol)
        self.mf.max_cycle = self.parameters.max_cycle
        self.mf.conv_tol = self.parameters.conv_tol

        # checkpoint file : default is /tmp
        if self.parameters.chkfile:
            self.mf.chkfile = self.parameters.chkfile

        # density_fit ( or resolution of identity (RI) approximation)
        if self.parameters.density_fit:
            if self.parameters.auxbasis == 'auto':
                self.mf.density_fit()
            elif self.parameters.auxbasis != 'auto':
                self.mf.density_fit(auxbasis=self.parameters.auxbasis)

        # Hartree-Fock calculation
        self.e_hf = self.mf.kernel()

        # post-HF correlaction
        self.e_corr = self.post_hf(self.mf).kernel()[0]

In [13]:
# from ase import Atoms

# # 물 분자 정의
# mol = Atoms('H2O', positions=[(0.0, 0.0, 0.0),
#                               (0.0, 0.0, 0.96),
#                               (0.0, 0.76, -0.24)])

In [25]:
%%writefile ethane.xyz
8
Ethane
  H      1.1851     -0.0039      0.9875
  C      0.7516     -0.0225     -0.0209
  H      1.1669      0.8330     -0.5693
  H      1.1155     -0.9329     -0.5145
  C     -0.7516      0.0225      0.0209
  H     -1.1669     -0.8334      0.5687
  H     -1.1157      0.9326      0.5151
  H     -1.1850      0.0044     -0.9875

Writing ethane.xyz


In [26]:
from ase.io import read

mol = read('ethane.xyz', format='xyz')

In [14]:
# default parameters
PySCFCalculator().parameters

{'charge': 0,
 'spin': 0,
 'method': 'mp2',
 'hf': 'rhf',
 'symmetry': False,
 'basis': 'def2-tzvp',
 'density_fit': True,
 'auxbasis': 'auto',
 'device': 'auto',
 'max_cycle': 50,
 'conv_tol': 1e-09,
 'verbose': 4,
 'max_memory': 150000,
 'chkfile': None,
 'output': None}

In [27]:
%%time

# mp2 / df
calc = PySCFCalculator()
calc.parameters.verbose = 0
calc.parameters.method = 'mp2'
calc.parameters.density_fit = True

mol.calc = calc
mol.get_potential_energy()

CPU times: user 7.6 s, sys: 168 ms, total: 7.77 s
Wall time: 7.58 s


-2167.3273301692716

In [28]:
%%time

# mp2
calc = PySCFCalculator()
calc.parameters.verbose = 0
calc.parameters.method = 'mp2'
calc.parameters.density_fit = False

mol.calc = calc
mol.get_potential_energy()

CPU times: user 10.7 s, sys: 256 ms, total: 11 s
Wall time: 10.8 s


-2167.3308455838014

In [29]:
%%time

# ccsd
calc = PySCFCalculator()
calc.parameters.verbose = 0
calc.parameters.method = 'ccsd'
calc.parameters.density_fit = False

mol.calc = calc
mol.get_potential_energy()

CPU times: user 27.2 s, sys: 1.55 s, total: 28.7 s
Wall time: 29 s


-2168.2984415137958

In [30]:
# ccsd / df --> Not Implemented
calc = PySCFCalculator()
calc.parameters.verbose = 0
calc.parameters.method = 'ccsd'
calc.parameters.density_fit = True

mol.calc = calc
mol.get_potential_energy()

NotImplementedError: gpu4pyscf.cc.ccsd_incore.CCSD does not support density_fit yet.